In [1]:
import pandas as pd
import myProject
from dash import Dash, html, dcc, callback, Output, Input, State
import pandas as pd
import plotly.express as px

In [3]:
nba_api_df = pd.read_csv('pic16b_nba_stats.csv')
sorted_fantasy_df = myProject.fantasy_stats(nba_api_df)

In [4]:
next_year_points = myProject.predict_next_fantasy_point(sorted_fantasy_df)

In [5]:
sorted_df = sorted_fantasy_df.copy()
merged_df = pd.merge(
    next_year_points[['PLAYER', 'TEAM', 'Pred Fantasy Points']],  
    sorted_df[['PLAYER', 'TEAM', 'Year','Fantasy Points']],  
    on=['PLAYER', 'TEAM'],  
    how='left'  
)

In [6]:
team_colors = {
    'ATL': '#E03A3E',  # Atlanta Hawks
    'BOS': '#007A33',  # Boston Celtics
    'BRK': '#000000',  # Brooklyn Nets
    'CHA': '#1D1160',  # Charlotte Hornets
    'CHI': '#CE1141',  # Chicago Bulls
    'CLE': '#6A1E1C',  # Cleveland Cavaliers
    'DAL': '#0061F2',  # Dallas Mavericks
    'DEN': '#0E76A8',  # Denver Nuggets
    'DET': '#044DDE',  # Detroit Pistons
    'GSW': '#006BB6',  # Golden State Warriors
    'HOU': '#CE1141',  # Houston Rockets
    'IND': '#FDBB30',  # Indiana Pacers
    'LAC': '#1D428A',  # Los Angeles Clippers
    'LAL': '#552583',  # Los Angeles Lakers
    'MEM': '#5D2F6E',  # Memphis Grizzlies
    'MIA': '#98002E',  # Miami Heat
    'MIL': '#00471B',  # Milwaukee Bucks
    'MIN': '#236192',  # Minnesota Timberwolves
    'NOP': '#002B5C',  # New Orleans Pelicans
    'NYK': '#DE7104',  # New York Knicks
    'OKC': '#D44904',  # Oklahoma City Thunder
    'ORL': '#0077C0',  # Orlando Magic
    'PHI': '#467CC2',  # Philadelphia 76ers
    'PHO': '#E56020',  # Phoenix Suns
    'POR': '#807471',  # Portland Trail Blazers
    'SAC': '#7A4BB1',  # Sacramento Kings
    'SAS': '#B5B5B5',  # San Antonio Spurs
    'TOR': '#AD2323',  # Toronto Raptors
    'UTH': '#EEfA0A',  # Utah Jazz
    'WAS': '#002F6C'   # Washington Wizards
}

In [7]:

#setting up our data 
filtered_df = sorted_fantasy_df.sort_values(by='Year')
df = filtered_df

#initialize the website
app = Dash()

#create the layout
app.layout = html.Div([
    html.H1("NBA Player Stats Dashboard", style={'text-align': 'center'}),
    html.Hr(),
    dcc.Dropdown(
        id='stat-dropdown',
        options=[
            {'label': col, 'value': col} for col in df.columns[4:]  #List options from FGM to Fantasy Points
        ],
        value='Fantasy Points',  #Default 
        style={'width': '50%'}
    ),
    html.Br(),
    dcc.RadioItems(
        options=['Single Player', 'Compare Players'],
        value='Single Player',
        id='mode-selector'
    ),
    html.Br(),
    html.Div([
        html.Label("Player Name:"),
        dcc.Input(id='player-name-input', value='DeMar DeRozan', type='text'),
        html.Label("Second Player Name:"),
        dcc.Input(id='second-player-name-input', value = 'Luka Dončić', type='text')
    ]),
    html.Br(),
    dcc.Graph(figure={}, id='player-fantasy-graph'), 
    html.Br(),
     html.Div([
        dcc.Markdown(
            """
            **Disclaimer:**  
            Fantalyzer is for informational and entertainment purposes only. 
            We are not responsible for any financial losses or decisions made based on the information provided. 
            Fantalyzer does not endorse or encourage gambling in any form.
            """
        ),
        html.Br(),
        html.P("© 2024 NBA Fantalyzer", style={'fontSize': '12px', 'textAlign': 'center'})
    ], style={'marginTop': '50px', 'padding': '10px', 'borderTop': '1px solid #ccc', 'fontSize': '14px', 'color': '#666'}),
    html.Br(),
    html.H2("Fantasy Trade Simulator", style={'text-align': 'center'}),
    html.Hr(),
    html.Br(),
    html.Div([
        html.Div([
            #Selecting team one
            html.Div([
                html.Label("Select Team 1"),
                dcc.Dropdown(
                    id='team1-dropdown',
                    options=[{'label': team, 'value': team} for team in df['TEAM'].unique()],
                    value='DAL',  
                    style={'display': 'inline-block', 'width': '100%'}
                )
            ], style={'width': '100%', 'display': 'inline-block'}), 
            html.Br(),
            #Selecting the players from team1
            html.Div([
                html.Label("Select Players from Team 1"),
                dcc.Checklist(
                    id='team1-players',
                    options=[],  
                    style={'width': '100%'}
                )
            ], style={'width': '100%', 'display': 'inline-block'})  

        ], style={'width': '48%', 'display': 'inline-block'}),  

        #Team2
        html.Div([
            #Dropdown
            html.Div([
                html.Label("Select Team 2"),
                dcc.Dropdown(
                    id='team2-dropdown',
                    options=[{'label': team, 'value': team} for team in df['TEAM'].unique()],
                    value='BOS',  
                    style={'display': 'inline-block', 'width': '100%'}
                )
            ], style={'width': '100%', 'display': 'inline-block'}),
            html.Br(),
            #Selecting team2
            html.Div([
                html.Label("Select Players from Team 2"),
                dcc.Checklist(
                    id='team2-players',
                    options=[],  
                    style={'width': '100%'}
                )
            ], style={'width': '100%', 'display': 'inline-block'}) 

        ], style={'width': '48%', 'display': 'inline-block'}),  

    ], style={'display': 'flex', 'justify-content': 'space-between', 'gap': '10px'}),  

    
    html.Br(),
    # Button to simulate trade
    html.Button("Simulate Prospective Trade", id='trade-button', style={'font-size':'20px',
                                                                        'padding':'15px 30px',
                                                                        'margin':'20px auto',
                                                                        'background-color':'#007BFF',
                                                                        'color':'white',
                                                                        'border-radius': '5px'}
    ),
    # Display team analysis
    html.H3("Original Team Analyis"),
    html.Div(id='team-analysis'),
    html.Div(id='trade-analysis'),
    dcc.Graph(id='team-graph'),
    html.Br(),
        html.Div([
        dcc.Markdown(
            """
            The shaded region represents the predicted team fantasy points of 2024-2025 based on the hypothetical trade,
            it is not representative of reality and is prone to errors due to the nature of our system of calculating fantasy points.
            It tends lower than it likely will be because we can't accurately calculate predictive values when there are only a few data points avaliable.
            """
        ),
    ], style={'marginTop': '50px', 'padding': '10px', 'borderTop': '1px solid #ccc', 'fontSize': '14px', 'color': '#666'}), 

])



# Callback to update the graph based on the selected stat and player inputs
@app.callback(
    Output('player-fantasy-graph', 'figure'),
    [Input('stat-dropdown', 'value'),
     Input('mode-selector', 'value'),
     Input('player-name-input', 'value'),
     Input('second-player-name-input', 'value')]
)

def update_graph(stat, mode, player_name, second_player_name):
    fig = px.bar(title="Enter player names to see their fantasy stats")
    
    #Single Player 
    if mode == 'Single Player' and player_name:
        filtered_df = df[df['PLAYER'].str.contains(player_name, case=False, na=False)]
        fig = px.bar(
            filtered_df,
            x='Year',
            y=stat,
            color='TEAM',
            title=f"{player_name}'s {stat} Over the Years",
            color_discrete_map=team_colors
        )
    
    #Compare Players
    elif mode == 'Compare Players' and player_name and second_player_name:
        filtered_df = df[df['PLAYER'].str.contains(player_name + '|' + second_player_name, case=False, na=False)]
        fig = px.bar(
            filtered_df,
            x='Year',
            y=stat,
            color='TEAM',
            barmode='group',
            title=f"Comparison of {player_name} and {second_player_name} {stat}", 
            color_discrete_map=team_colors
        )

    return fig

#call back for the teams displaying the players of the current year 
@app.callback(
    [Output('team1-players', 'options'),
     Output('team2-players', 'options'),
    Output('team-analysis', 'children')],
    [Input('team1-dropdown', 'value'),
     Input('team2-dropdown', 'value')]
)
def update_players(team1, team2):

    current_df = df[df['Year'] == '2023-24']

    #capturing the list of players in a variable
    team1_players = current_df[current_df['TEAM'] == team1]['PLAYER'].unique()
    team2_players = current_df[current_df['TEAM'] == team2]['PLAYER'].unique()
    
    performance1 = myProject.team_analysis(current_df, team1)
    performance2 = myProject.team_analysis(current_df, team2)
    
    words = [html.Div([html.P(f"{team1}'s team data: {performance1}"),
                        html.P(f"{team2}'s team data: {performance2}")
                            ])]

    # returning the options and analysis
    return [{'label': player, 'value': player} for player in team1_players], \
           [{'label': player, 'value': player} for player in team2_players], \
            words


#Callback to update button
@app.callback(
    Output('trade-button', 'children'),
    [Input('team1-players', 'value'),
     Input('team2-players', 'value')]
)

def button_label(team1_players, team2_players):
    if team1_players and team2_players:
        return f"Trade {len(team1_players)} players from Team 1 with {len(team2_players)} players from Team 2"
    else:
        return "Trade Selected Players"
    
#Callback to simulate trade through the button
@app.callback(
    [Output('trade-analysis', 'children'),
    Output('team-graph','figure')],
    [Input('trade-button', 'n_clicks')],
     [State('team1-players', 'value'),
     State('team2-players', 'value'),
    State('team1-dropdown', 'value'),
     State('team2-dropdown', 'value')]
     )

def simulate_trade_and_graph(n_clicks, team1_players, team2_players, team1, team2):
    if n_clicks and team1_players and team2_players:
        #Swap the teams for the selected players
        df.loc[df['PLAYER'].isin(team1_players), 'TEAM'] = team2
        df.loc[df['PLAYER'].isin(team2_players), 'TEAM'] = team1
        
        new_performance1 = myProject.team_analysis(df, team1)
        new_performance2 = myProject.team_analysis(df, team2)

        show = [html.Div([
            html.H3('Trade Analysis'),
            html.P(f"{team1}'s team data: {new_performance1}"),
            html.P(f"{team2}'s team data: {new_performance2}")
            ])]
        
        predictive_df = myProject.team_graph_df(df, merged_df,team1,team2)
    
        #Melt the DataFrame so that the values can be on one line
        melted_df = pd.melt(predictive_df, id_vars=['Year'], value_vars=['Team1 Total Fantasy Points', 'Team2 Total Fantasy Points'],
                            var_name='TEAM', value_name='Total Fantasy Points')
        
        melted_df['TEAM'] = melted_df['TEAM'].replace({
            'Team1 Total Fantasy Points': team1,  
            'Team2 Total Fantasy Points': team2   
        })
        
        # Create a line graph using Plotly Express
        fig = px.line(melted_df, x='Year', y='Total Fantasy Points', 
                      labels={'Year': 'Year', 'Total Fantasy Points': 'Total Fantasy Points'},
                      color='TEAM',
                      title=f'Total Fantasy Points Over Time: {team1} vs {team2}',
                     color_discrete_map=team_colors,
                     markers=True)
        
        # Add a shaded region for the predicted year (2024-25)
        fig.add_shape(
            type="rect",  
            x0="2023-24", 
            x1="2024-25",  
            y0=0,  
            y1=1, 
            xref="x",  
            yref="paper",
            fillcolor="rgba(0, 100, 255, 0.2)"
        )
        
        return show, fig
    
    else:
        no_show = [html.Div([
        html.H3('Trade Analysis'),
        html.P('No trade executed yet.')
    ])]
        empty_figure = px.line()
        return no_show, empty_figure   

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port = 8501)